# Import Required Libraries
Import the necessary libraries, including Azure Kinect SDK, OpenCV, and Detectron2.

In [ ]:
# Import Azure Kinect SDK
from pykinect_azure.k4a import PyK4A, Config, ImageFormat

# Import OpenCV for video processing
import cv2

# Import Detectron2 for panoptic segmentation
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Import additional libraries
import torch
import numpy as np
import matplotlib.pyplot as plt

# Configure Azure Kinect
Set up the Azure Kinect device to start receiving a video stream.

In [ ]:
# Configure Azure Kinect
# Set up the Azure Kinect device to start receiving a video stream.

# Initialize the Azure Kinect device
k4a = PyK4A(Config(color_format=ImageFormat.COLOR_BGRA32, color_resolution=720, depth_mode='NFOV_UNBINNED', synchronized_images_only=True))
k4a.start()

# Function to capture video frames from Azure Kinect
def get_video_frame():
    capture = k4a.get_capture()
    if capture.color is not None:
        return capture.color[:, :, :3]  # Return the color image
    else:
        return None

# Test capturing a frame
frame = get_video_frame()
if frame is not None:
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGRA2RGB))
    plt.title("Sample Frame from Azure Kinect")
    plt.show()
else:
    print("Failed to capture a frame from Azure Kinect.")

# Receive Video Stream
Capture the video stream from the Azure Kinect device using OpenCV.

In [ ]:
# Receive Video Stream
# Capture the video stream from the Azure Kinect device using OpenCV.

# Function to continuously capture video frames from Azure Kinect
def capture_video_stream():
    while True:
        frame = get_video_frame()
        if frame is not None:
            # Display the frame
            cv2.imshow('Azure Kinect Video Stream', frame)
            
            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Failed to capture a frame from Azure Kinect.")
            break

    # Release resources
    cv2.destroyAllWindows()

# Start capturing the video stream
capture_video_stream()

# Load Panoptic Segmentation Model
Load the Detectron2 panoptic segmentation model and configure it to use the GPU.

In [ ]:
# Load Panoptic Segmentation Model
# Load the Detectron2 panoptic segmentation model and configure it to use the GPU.

# Set up Detectron2 configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml")
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available, otherwise CPU

# Create the predictor
predictor = DefaultPredictor(cfg)

# Test the model on a sample frame
if frame is not None:
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.title("Panoptic Segmentation on Sample Frame")
    plt.show()
else:
    print("No frame available for testing the model.")

# Apply Model on Video Stream
Apply the panoptic segmentation model on each frame of the video stream. Adjust the framerate if the model is too slow.

In [ ]:
# Apply Model on Video Stream
# Apply the panoptic segmentation model on each frame of the video stream. Adjust the framerate if the model is too slow.

# Function to apply the panoptic segmentation model on video frames
def apply_model_on_video_stream():
    while True:
        frame = get_video_frame()
        if frame is not None:
            # Apply the model on the frame
            outputs = predictor(frame)
            
            # Visualize the original frame and the panoptic segmentation result
            v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            panoptic_frame = out.get_image()[:, :, ::-1]
            
            # Concatenate the original frame and the panoptic frame side by side
            combined_frame = np.hstack((frame, panoptic_frame))
            
            # Display the combined frame
            cv2.imshow('Original and Panoptic Segmentation', combined_frame)
            
            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Failed to capture a frame from Azure Kinect.")
            break

    # Release resources
    cv2.destroyAllWindows()

# Start applying the model on the video stream
apply_model_on_video_stream()

# Visualize Original and Segmented Video
Create a window to display the original video and the segmented video side by side or top-bottom using OpenCV.

In [ ]:
# Visualize Original and Segmented Video
# Create a window to display the original video and the segmented video side by side or top-bottom using OpenCV.

# Function to visualize the original and segmented video frames
def visualize_original_and_segmented_video():
    while True:
        frame = get_video_frame()
        if frame is not None:
            # Apply the model on the frame
            outputs = predictor(frame)
            
            # Visualize the original frame and the panoptic segmentation result
            v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            panoptic_frame = out.get_image()[:, :, ::-1]
            
            # Concatenate the original frame and the panoptic frame side by side
            combined_frame = np.hstack((frame, panoptic_frame))
            
            # Display the combined frame
            cv2.imshow('Original and Panoptic Segmentation', combined_frame)
            
            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Failed to capture a frame from Azure Kinect.")
            break

    # Release resources
    cv2.destroyAllWindows()

# Start visualizing the original and segmented video
visualize_original_and_segmented_video()